In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('/kaggle/input/inflation-in-indonesia-per-month-from-2003-2023/Inflasi Dari BI.xlsx')
df


,No,Tanggal,Data Inflasi (dalam persen)
0,1,2003-01-01,8.68
1,2,2003-02-01,7.60
2,3,2003-03-01,7.17
3,4,2003-04-01,7.62
4,5,2003-05-01,7.15
...,...,...,...
240,241,2023-01-01,5.28
241,242,2023-02-01,5.47
242,243,2023-03-01,4.97
243,244,2023-04-01,4.33


In [3]:
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
reference_date = pd.to_datetime('2003-01-01')
df['Tanggal Referensi'] = (df['Tanggal']-reference_date).dt.days
df

,No,Tanggal,Data Inflasi (dalam persen),Tanggal Referensi
0,1,2003-01-01,8.68,0
1,2,2003-02-01,7.60,31
2,3,2003-03-01,7.17,59
3,4,2003-04-01,7.62,90
4,5,2003-05-01,7.15,120
...,...,...,...,...
240,241,2023-01-01,5.28,7305
241,242,2023-02-01,5.47,7336
242,243,2023-03-01,4.97,7364
243,244,2023-04-01,4.33,7395


In [4]:
df_train = df[['Tanggal Referensi', 'Data Inflasi (dalam persen)']]
df_train

,Tanggal Referensi,Data Inflasi (dalam persen)
0,0,8.68
1,31,7.60
2,59,7.17
3,90,7.62
4,120,7.15
...,...,...
240,7305,5.28
241,7336,5.47
242,7364,4.97
243,7395,4.33


In [5]:
df_train.corr()

,Tanggal Referensi,Data Inflasi (dalam persen)
Tanggal Referensi,1.000000,-0.609004
Data Inflasi (dalam persen),-0.609004,1.000000


In [6]:
df_train.columns

Index(['Tanggal Referensi', 'Data Inflasi (dalam persen)'], dtype='object')

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Pisahkan fitur dan target
X = df[['Tanggal Referensi']]
y = df['Data Inflasi (dalam persen)']

# Bagi data menjadi 80% data pelatihan dan 20% data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True)

# Inisialisasi dan latih model Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Lakukan prediksi menggunakan data uji
y_pred_rf = rf_model.predict(X_test)

# Evaluasi model Random Forest Regressor
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Regressor:")
print("Mean Squared Error (MSE):", mse_rf)
print("Mean Absolute Error (MAE):", mae_rf)
print("R2 Score:", r2_rf)

# Inisialisasi dan latih model Decision Tree Regressor
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

# Lakukan prediksi menggunakan data uji
y_pred_tree = tree_model.predict(X_test)

# Evaluasi model Decision Tree Regressor
mse_tree = mean_squared_error(y_test, y_pred_tree)
mae_tree = mean_absolute_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)

print("\nDecision Tree Regressor:")
print("Mean Squared Error (MSE):", mse_tree)
print("Mean Absolute Error (MAE):", mae_tree)
print("R2 Score:", r2_tree)

# Inisialisasi model XGBoost
xgb_model = XGBRegressor()

# Latih model dengan data pelatihan
xgb_model.fit(X_train, y_train)

# Lakukan prediksi menggunakan data uji
y_pred_xgb = xgb_model.predict(X_test)

# Evaluasi model XGBoost
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("\nXGBoost Regressor:")
print("Mean Squared Error (MSE):", mse_xgb)
print("Mean Absolute Error (MAE):", mae_xgb)
print("R2 Score:", r2_xgb)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Random Forest Regressor:
Mean Squared Error (MSE): 1.674575073265307
Mean Absolute Error (MAE): 0.6036142857142864
R2 Score: 0.8408765354196029

Decision Tree Regressor:
Mean Squared Error (MSE): 3.446130612244899
Mean Absolute Error (MAE): 0.8191836734693879
R2 Score: 0.6725376776642755

XGBoost Regressor:
Mean Squared Error (MSE): 3.412384901773745
Mean Absolute Error (MAE): 0.7927371493164374
R2 Score: 0.6757443027064282


In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Pisahkan fitur dan target
X = df[['Tanggal Referensi']]
y = df['Data Inflasi (dalam persen)']

# Bagi data menjadi 80% data pelatihan dan 20% data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Inisialisasi model Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)

# Tentukan hyperparameter yang ingin dituning dan daftar nilai-nilai yang akan diuji
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 2, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inisialisasi GridSearchCV dengan model, hyperparameter yang ingin dituning, dan metrik evaluasi 'r2'
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=10, scoring='r2')

# Latih model dengan GridSearchCV
grid_search_rf.fit(X_train, y_train)

# Dapatkan model terbaik setelah tuning
best_rf_model = grid_search_rf.best_estimator_

# Evaluasi model Random Forest Regressor
y_pred_rf = best_rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Regressor:")
print("Best Model Hyperparameters:", grid_search_rf.best_params_)
print("Mean Squared Error (MSE):", mse_rf)
print("Mean Absolute Error (MAE):", mae_rf)
print("R2 Score:", r2_rf)


Random Forest Regressor:
Best Model Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Mean Squared Error (MSE): 1.8124416138775523
Mean Absolute Error (MAE): 0.6179632653061226
R2 Score: 0.8277760170002302


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Pisahkan fitur dan target
X = df[['Tanggal Referensi']]
y = df['Data Inflasi (dalam persen)']

# Bagi data menjadi 80% data pelatihan dan 20% data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Inisialisasi model Decision Tree Regressor
tree_model = DecisionTreeRegressor(random_state=42)

# Latih model Decision Tree Regressor
tree_model.fit(X_train, y_train)

# Evaluasi model Decision Tree Regressor
y_pred_tree = tree_model.predict(X_test)
mse_tree = mean_squared_error(y_test, y_pred_tree)
mae_tree = mean_absolute_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)

print("\nDecision Tree Regressor:")
print("Mean Squared Error (MSE):", mse_tree)
print("Mean Absolute Error (MAE):", mae_tree)
print("R2 Score:", r2_tree)



Decision Tree Regressor:
Mean Squared Error (MSE): 3.446130612244899
Mean Absolute Error (MAE): 0.8191836734693879
R2 Score: 0.6725376776642755


In [10]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Pisahkan fitur dan target
X = df[['Tanggal Referensi']]
y = df['Data Inflasi (dalam persen)']

# Bagi data menjadi 80% data pelatihan dan 20% data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Inisialisasi model XGBoost
xgb_model = XGBRegressor()

# Latih model XGBoost
xgb_model.fit(X_train, y_train)

# Evaluasi model XGBoost
y_pred_xgb = xgb_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("\nXGBoost Regressor:")
print("Mean Squared Error (MSE):", mse_xgb)
print("Mean Absolute Error (MAE):", mae_xgb)
print("R2 Score:", r2_xgb)



XGBoost Regressor:
Mean Squared Error (MSE): 3.412384901773745
Mean Absolute Error (MAE): 0.7927371493164374
R2 Score: 0.6757443027064282


In [11]:
df_new = pd.read_excel('/kaggle/input/inflation-in-indonesia-per-month-from-2003-2023/Inflasi Dari BI.xlsx')

# Define a function to apply the desired styles
def style_dataframe(df_new):
    # Define the CSS styles
    styles = [
        {'selector': 'th', 'props': [('background-color', '#BF753A'), ('color', 'white')]},
        {'selector': 'td', 'props': [('background-color', '#E4AD82'), ('color', 'black')]},
        {'selector': 'caption', 'props': [('caption-side', 'bottom')]}
    ]
    
    # Create the Styler object and apply the styles
    styled_df = df_new.style.set_table_styles(styles)
    
    return styled_df

# Apply styles to the DataFrame
styled_df = style_dataframe(df_new)

# Display the styled DataFrame
styled_df

,No,Tanggal,Data Inflasi (dalam persen)
0,1,2003-01-01 00:00:00,8.680000
1,2,2003-02-01 00:00:00,7.600000
2,3,2003-03-01 00:00:00,7.170000
3,4,2003-04-01 00:00:00,7.620000
4,5,2003-05-01 00:00:00,7.150000
5,6,2003-06-01 00:00:00,6.980000
6,7,2003-07-01 00:00:00,6.270000
7,8,2003-08-01 00:00:00,6.510000
8,9,2003-09-01 00:00:00,6.330000
9,10,2003-10-01 00:00:00,6.480000
